In [56]:
## If running on Google Colab, run this cell to mount Google Drive to access files on Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google'

This jupyter notebook will develp the SOP for updating the Monday.com studies board consistently.

In [7]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.0 MB/s eta 0:00:00


In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import re

In [2]:
## Set this to the directory where:
## 1- Every group on Monday Studies board has been exported as an individual group to
## 2- All relevant tables from MySql database for HEAL have been exported as a csv to.
input_dir = Path("/Users/hinashah/Documents/HEAL/MondayUpdate_Feb2025")

In [3]:
def get_unique_values(df:pd.DataFrame, col_name:str='appl_id'):
    if col_name in df.columns:
        return df[ ~pd.isna(df[col_name])][col_name].drop_duplicates()
    return None

def get_na_count(df:pd.DataFrame, col_name:str='appl_id'):
    if col_name in df.columns:
        return len(df[pd.isna(df[col_name])])
    return -1

In [4]:
gt_file = pd.read_csv(input_dir/"study_lookup_table.csv", dtype=str)
gt_file.replace("0", np.nan, inplace=True)

print(len(gt_file))
print(gt_file.columns)
print(gt_file.dtypes)
### QC the lookup table:
for k in gt_file.columns:
    print(f"Number of distinct values in --{k}--: {len(get_unique_values(gt_file, k))}")
    print(f"---- NA count: {get_na_count(gt_file, k)}")
    ## Look for patterns?
    if 'appl' in k:
        d = gt_file[[ (not pd.isna(l)) and (not l.isdigit()) for l in gt_file[k] ]]
        print(f"Number of funky looking appl_ids: {len(d)}")
    elif k == 'study_hdp_id':
        d = gt_file[ [ (not pd.isna(l)) and (re.match(r'HDP[\d]+', l) is None) for l in gt_file[k]]]
        print(f" Number of funky looking HDPIDs: {len(d)}")

1823
Index(['appl_id', 'xstudy_id', 'study_most_recent_appl', 'study_hdp_id',
       'study_hdp_id_appl'],
      dtype='object')
appl_id                   object
xstudy_id                 object
study_most_recent_appl    object
study_hdp_id              object
study_hdp_id_appl         object
dtype: object
Number of distinct values in --appl_id--: 1808
---- NA count: 0
Number of funky looking appl_ids: 0
Number of distinct values in --xstudy_id--: 1434
---- NA count: 0
Number of distinct values in --study_most_recent_appl--: 1419
---- NA count: 0
Number of funky looking appl_ids: 0
Number of distinct values in --study_hdp_id--: 1330
---- NA count: 118
 Number of funky looking HDPIDs: 0
Number of distinct values in --study_hdp_id_appl--: 1315
---- NA count: 118
Number of funky looking appl_ids: 0


In [5]:
board_files_list = list(input_dir.glob("HEAL_Studies_*.xlsx"))
df = pd.read_excel(board_files_list[0], skiprows=4)
monday_board = df[~(df.Name.isin(['Studies under investigation', 'Name', 'Studies Not Added to Platform', 'CTN Protocols']))]
monday_board = monday_board[~pd.isna(monday_board['Name'])]

In [7]:
### Steps for updating Monday board:

## From Study lookup table, get unique set of most_recent_appl, study_hdp_id, and study_hdp_id_appl
lookup_fields = gt_file[['study_hdp_id', 'study_most_recent_appl', 'study_hdp_id_appl']].copy(deep=True).drop_duplicates()
## Create a column "Key" that will either have study_hdp_id OR most_recent_appl when study_hdp_id is empty
lookup_fields['key'] = [m if pd.isna(h) else h for (h, m) in lookup_fields[['study_hdp_id', 'study_most_recent_appl']].values ]

### A few checks:
## How many of the "keys" from Monday board are in lookup fields?
print(f"Number records from Monday already in lookup table: {len(monday_board[monday_board.Name.isin(lookup_fields.key)])}")
## How many of the keys from MOnday board are not there in looup fields
mondayboard_missingin_lookup = monday_board[~monday_board.Name.isin(lookup_fields.key)]
print(f"Number records from Monday that are not in lookup table (Consider these as discrepancies **Investigate**): {len(mondayboard_missingin_lookup)}")
## How many of the keys from lookup fields are not there in Monday??
lookup_missingin_mondayboard = lookup_fields[~lookup_fields.key.isin(monday_board.Name)]
print(f"Number records from lookup table that are not on Monday (Potentially new entries): {len(lookup_missingin_mondayboard)}")


Number records from Monday already in lookup table: 1419
Number records from Monday that are not in lookup table (Consider these as discrepancies **Investigate**): 44
Number records from lookup table that are not on Monday (Potentially new entries): 15


In [8]:
mondayboard_missingin_lookup

,Name,Most Recent Appl_ID,HDP appl_ID,Project #,Archived,HEAL-Related,Research Focus,Research Program,Title,Contact PI,...,"""Get the Data"" Engagement Board",Data Engagement,Repo per Platform,Platform Reg Time,CEDAR Form %,Creation Log,study_type,DD Tracker,Checklist Exempt,Do not Engage
1319,HDP01285,-,-,CTN-0130,n,N,-,-,-,-,...,Margaret Wadle (c/o Randy Boley),NaN,NAHDAP,2024-08-23 00:00:00,73.1,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N
1320,HDP01286,-,-,CTN-0133,n,N,-,-,-,-,...,CTN - Kimberly Page/Janet Page-Reeves,NaN,-,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,HDPID01286,N,N
1321,HDP01287,-,-,CTN-0095A2,n,N,-,-,-,-,...,CTN - Gavin Bart,NaN,NIDA Data Share,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,CTN0095A2-Data-Dictionary.xlsx,N,N
1322,HDP01288,-,-,CTN-0093,n,N,-,-,-,-,...,NaN,NaN,NIDA Data Share,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,CTN0093-DataDictionary.xlx,N,N
1323,HDP01289,-,-,CTN-0147,n,N,-,-,-,-,...,NaN,NaN,-,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N
1324,HDP01290,-,-,CTN-0099,n,N,-,-,-,-,...,CTN0099- Gail D'Onofrio,NaN,NIDA Data Share,2024-08-15 00:00:00,76.9,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N
1325,HDP01291,-,-,CTN-0088,n,N,-,-,-,-,...,CTN - Richard Schottenfeld,NaN,-,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N
1326,HDP01292,-,-,CTN-0144,n,N,-,-,-,-,...,CTN - Jose Szapocznik,NaN,-,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N
1327,HDP01293,-,-,CTN-0099-A-1,n,N,-,-,-,-,...,NaN,NaN,-,NaN,7.7,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N
1328,HDP01294,-,-,CTN-0095,n,N,-,-,-,-,...,NaN,NaN,NIDA Data Share,2024-08-20 00:00:00,78.8,"Hina Shah Jun 26, 2024 2:05 PM",CTN,NaN,N,N


In [9]:
# Get rest of the tables
convert_dict = {'appl_id':str}

awards_df = pd.read_csv(input_dir/"awards.csv", low_memory=False, dtype=convert_dict)
awards_df = awards_df.dropna(how='all')
print(f"Awards table has: {len(awards_df)} entries, with {len(get_unique_values(awards_df))} appl_ids")
reporter_df = pd.read_csv(input_dir/"reporter.csv", low_memory=False, dtype=convert_dict)
reporter_df = reporter_df.dropna(how='all')
print(f"Reporter table has: {len(reporter_df)} entries, with {len(get_unique_values(reporter_df))} appl_ids")
progress_tracker_df = pd.read_csv(input_dir/"progress_tracker.csv", low_memory=False, dtype=convert_dict)
print(f"Platform generated table has: {len(progress_tracker_df)} entries, with {len(get_unique_values(progress_tracker_df))} appl_ids")
print(f"Platform table has {len(get_unique_values(progress_tracker_df))} unique HDP IDs")
pi_emails_df = pd.read_csv(input_dir/"pi_emails.csv", low_memory=False, dtype=convert_dict)
print(f"Repo mapping table has: {len(pi_emails_df)} entrie, with {len(get_unique_values(pi_emails_df))} appl_ids")
resnet_df = pd.read_csv(input_dir/"research_networks.csv", low_memory=False, dtype=convert_dict)
print(f"Research Network table has: {len(resnet_df)} entrie, with {len(get_unique_values(resnet_df))} appl_ids")
engagement_flags_df = pd.read_csv(input_dir/"engagement_flags.csv", low_memory=False, dtype=convert_dict)
print(f"Engagment Flags table has: {len(resnet_df)} entrie, with {len(get_unique_values(engagement_flags_df))} appl_ids")


Awards table has: 2015 entries, with 2015 appl_ids
Reporter table has: 2015 entries, with 2015 appl_ids
Platform generated table has: 1494 entries, with 1440 appl_ids
Platform table has 1440 unique HDP IDs
Repo mapping table has: 1326 entrie, with 1326 appl_ids
Research Network table has: 2015 entrie, with 2015 appl_ids
Engagment Flags table has: 2015 entrie, with 1807 appl_ids


In [10]:
## Manipulate emails to carry forward emails from a previous appl_id to the most recent one according to the lookup table and email table
appl_ids = gt_file[['appl_id', 'study_most_recent_appl']].drop_duplicates()
appl_ids_emails = pd.merge(appl_ids, pi_emails_df, how='left', on='appl_id')

most_recent_emails = appl_ids_emails[ ~pd.isna(appl_ids_emails.pi_email)][['study_most_recent_appl', 'pi_email']].drop_duplicates()
most_recent_emails.rename(columns={'pi_email':'pi_email_latest'}, inplace=True)
print(f"ALL PI emails associated with a project (identified by most_recent_appl)\n {most_recent_emails}")
email_counts = most_recent_emails.groupby('study_most_recent_appl').size()
appl_ids_counts = appl_ids_emails.groupby('study_most_recent_appl').size()
print("Statistics on number of emails per project:: ")
print(email_counts.describe())

appl_ids_emails['email_count'] = [email_counts[k] if k in email_counts else 0 for k in appl_ids_emails['study_most_recent_appl']]
appl_ids_emails['applid_count'] = [appl_ids_counts[k] if k in appl_ids_counts else 0 for k in appl_ids_emails['study_most_recent_appl']]
appl_ids_emails['pi_email'] = appl_ids_emails['pi_email'].fillna('')
appl_ids_emails['keep'] = [1 if (c==0 or (c==1 and len(e)>0) or (c>1 and a==m)) else 0 for (c,a,m,e) in appl_ids_emails[['email_count', 'appl_id', 'study_most_recent_appl', 'pi_email' ]].values]

pi_emails_df_updated = appl_ids_emails[appl_ids_emails['keep']==1][['study_most_recent_appl', 'pi_email']].drop_duplicates()
pi_emails_df_updated['pi_email'] = [k.strip() for k in pi_emails_df_updated['pi_email']]
print("Emails that were kept for each project when there's only one email available, or when email is available for the most recent award (study_most_recent_appl)")
print(pi_emails_df_updated)

## Get Monday board emails, and fill in any that are different from mysql..
pi_emails_df_updated_monday = pd.merge(pi_emails_df_updated, monday_board[['Most Recent Appl_ID', 'Contact Email']].drop_duplicates(), how='left', left_on='study_most_recent_appl', right_on='Most Recent Appl_ID').drop(columns='Most Recent Appl_ID')
pi_emails_df_updated_monday['Contact Email'] = pi_emails_df_updated_monday['Contact Email'].replace('-', '')
pi_emails_df_updated_monday['Contact Email'] = pi_emails_df_updated_monday['Contact Email'].fillna('-')
pi_emails_df_updated_monday['pi_email_updated'] = [me if (len(e)==0 and len(me) > 1) else e for (e,me) in pi_emails_df_updated_monday[['pi_email', 'Contact Email']].values]
print("Adding any Monday Board emails when emails missing from MySql")
print(pi_emails_df_updated_monday)
pi_emails_df_updated_monday.to_csv(input_dir/"email_updates.csv", index=False)
appl_ids_emails.to_csv(input_dir/"email_counts.csv", index=False)

pi_emails_df_updated = pi_emails_df_updated_monday[['study_most_recent_appl', 'pi_email_updated']].rename(columns={'pi_email_updated':'pi_email'})

ALL PI emails associated with a project (identified by most_recent_appl)
      study_most_recent_appl               pi_email_latest
1                   9755001             kwatkins@rand.org
2                   9850412               damico@rand.org
3                  10996795         acfernan@med.umich.ed
4                  11001407               fqeadan@luc.edu
5                   9869480             LYNN.DEBAR@KP.ORG
...                     ...                           ...
1799               11141362        rredwards@partners.org
1802               11193588                sreis@pitt.edu
1803               11193636         monica.kraft@mssm.edu
1804               11195200  lucila.ohno-machado@yale.edu
1805               11193592        szuchner@med.miami.edu

[1124 rows x 2 columns]
Statistics on number of emails per project:: 
count    1101.000000
mean        1.020890
std         0.143081
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         

In [11]:
## Collect fields from report/awards tables that are required by Monday Board
rename_dict = {'proj_num':'Project #',
               'proj_title':'Title',
                'rfa':'Research Focus',
                'res_prg':'Research Program',
                'ctc_pi_nm':'Contact PI',
                'pi_email':'Contact Email',
                'adm_ic':'Administering IC',
                'prg_ofc':'NIH PO',
                'org_nm': 'Institution(s)',
                'pi':'PI(s)',
                'org_cy':'City',
                'org_st':'State',
                'act_code':'Activity Code',
                'awd_ty':'Award Type',
                'fisc_yr':'Award Year',
                'tot_fund':'Total Funded',
                'proj_abs':'Summary',
                'fund_mech': 'SBIR/STTR',
                'proj_strt_date':'Project Start',
                'proj_end_date':'Project End',
                'proj_url':'Reporter Link',
                'res_net':'Research Network',
                'time_of_registration':'Platform Reg Time',
                'overall_percent_complete':'CEDAR Form %',
                'repository_name' : 'Repo per Platform',
                'archived':'Archived',
                'heal_funded':'HEAL-Related',
                'do_not_engage':'Do not Engage',
                'checklist_exempt_all':'Checklist Exempt'
                }

def create_mysql_subset(in_df:pd.DataFrame, extra_fields = ['appl_id']):
    subset = in_df[[k for k in rename_dict.keys() if k in in_df.columns] + extra_fields].copy(deep=True)
    subset.rename(columns={k:v for k,v in rename_dict.items() if k in in_df.columns}, inplace=True)
    return subset

mysql_fields_reporter = create_mysql_subset(awards_df)
mysql_fields_awards = create_mysql_subset(reporter_df)
mysql_fields_platform = create_mysql_subset(progress_tracker_df, extra_fields=['hdp_id'])
mysql_fields_piemails = create_mysql_subset(pi_emails_df_updated, extra_fields=['study_most_recent_appl'])
mysql_fields_resnet = create_mysql_subset(resnet_df)
mysql_fields_resnet['Research Network'] = [k.upper() if not pd.isna(k) else '' for k in mysql_fields_resnet['Research Network']]
mysql_fields_enagementflags = create_mysql_subset(engagement_flags_df)

In [12]:
## Combine all the fields into one table using "Most Recent Appl_ID" as the key. Monday Board will display information from the most recent appl_id for a project, which is available in mysql's study lookup table.
print(len(lookup_fields))
data_merge_1 = pd.merge(lookup_fields, mysql_fields_reporter, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_1))
data_merge_2 = pd.merge(data_merge_1, mysql_fields_awards, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_2))
data_merge_1 = pd.merge(data_merge_2, mysql_fields_platform, how='left', left_on='study_hdp_id', right_on='hdp_id')
print(len(data_merge_1))
data_merge_2 = pd.merge(data_merge_1, mysql_fields_resnet, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_2))
data_merge_1 = pd.merge(data_merge_2, mysql_fields_enagementflags, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_1))
combined_data_ph1 = pd.merge(data_merge_1, mysql_fields_piemails, how='left', on='study_most_recent_appl')
print(len(combined_data_ph1))
print(len(combined_data_ph1.drop_duplicates()))


1434
1434
1434
1434
1434
1434
1434
1434


In [13]:
# Fill in holes in the mysql data using the progress tracker data a.k.a platform MDS data.
rename_dict = {'project_num':'Project #',
               'project_title':'Title',
                'investigators_name':'PI(s)',
                'award_type':'Award Type',
                'year_awarded':'Award Year',
                'award_amount':'Total Funded',
                'study_name':'Summary',
                'project_end_date':'Project End',
                'nih_reporter_link':'Reporter Link',
                'time_of_registration':'Platform Reg Time',
                'overall_percent_complete':'CEDAR Form %',
                'repository_name' : 'Repo per Platform',
                'archived':'Archived',
                }
progress_tracker_data = progress_tracker_df.copy(deep=True)
progress_tracker_data['project_title'] = progress_tracker_data['project_title'].replace('0', '')
progress_tracker_data = create_mysql_subset(progress_tracker_data, extra_fields=['hdp_id'])

progress_tracker_data['PI(s)'] = progress_tracker_data['PI(s)'].fillna('')
progress_tracker_data['PI(s)'] = [ k.translate(str.maketrans(',', ';', "[]\'")) for k in  progress_tracker_data['PI(s)']]

progress_tracker_data['key'] = progress_tracker_data['hdp_id']
progress_tracker_data['study_hdp_id'] = progress_tracker_data['hdp_id']
progress_tracker_data['Research Network'] = [ 'CTN' if k.startswith('CTN') else '' for k in progress_tracker_data['Project #']]

fill_in_data = pd.merge(combined_data_ph1, progress_tracker_data, how='left', on='study_hdp_id')
fill_in_data.to_csv(input_dir/"tmp.csv", index=False)

columns_to_compare = list(rename_dict.values())
columns_to_compare.extend(['key', 'Research Network'])

for k in columns_to_compare:
    k_x = k+'_x'
    k_y = k+'_y'
    fill_in_data[k] = [v_y if pd.isna(v_x) else v_x for (v_x, v_y) in fill_in_data[[k_x, k_y]].values]
    fill_in_data.drop(columns=[k_x, k_y], inplace=True)

columns = fill_in_data.columns.sort_values()
fill_in_data = fill_in_data[columns]
fill_in_data.to_csv(input_dir/"tmp.csv", index=False)
combined_data_ph1 = fill_in_data.copy(deep=True)


In [14]:
## Add CTN  data
ctn_data = progress_tracker_df[[k.startswith('CTN') for k in progress_tracker_df['project_num']]]
ctn_data['project_title'] = ctn_data['project_title'].replace('0', '')
print(f"Number of CTN entries found in Platform MDS {len(ctn_data)}")
rename_dict = {'project_num':'Project #',
               'project_title':'Title',
                'investigators_name':'PI(s)',
                'award_type':'Award Type',
                'year_awarded':'Award Year',
                'award_amount':'Total Funded',
                'study_name':'Summary',
                'proj_end_date':'Project End',
                'nih_reporter_link':'Reporter Link',
                'time_of_registration':'Platform Reg Time',
                'overall_percent_complete':'CEDAR Form %',
                'repository_name' : 'Repo per Platform',
                'archived':'Archived',
                }
ctn_fields_platform = create_mysql_subset(ctn_data, extra_fields=['hdp_id'])
## Edit pi name
ctn_fields_platform['PI(s)'] = ctn_fields_platform['PI(s)'].fillna('')
ctn_fields_platform['PI(s)'] = [ k.translate(str.maketrans(',', ';', "[]\'")) for k in  ctn_fields_platform['PI(s)']]

ctn_fields_platform['key'] = ctn_fields_platform['hdp_id']
ctn_fields_platform['study_hdp_id'] = ctn_fields_platform['hdp_id']
ctn_fields_platform['Research Network'] = ['CTN']*len(ctn_fields_platform)

## Combine the data to the other data set
all_data = pd.concat([combined_data_ph1, ctn_fields_platform])

Number of CTN entries found in Platform MDS 40


/var/folders/tb/c_qhpk1j2jj3knzfnw72yj080000gq/T/ipykernel_16884/3796386117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctn_data['project_title'] = ctn_data['project_title'].replace('0', '')


In [15]:
print("------------ Preview of the final combined dataset ---------------")
all_data

------------ Preview of the final combined dataset ---------------


,Activity Code,Administering IC,Archived,Award Type,Award Year,CEDAR Form %,Checklist Exempt,City,Contact Email,Contact PI,...,Summary,Title,Total Funded,hdp_id_x,hdp_id_y,key,study_hdp_id,study_hdp_id_appl,study_most_recent_appl,hdp_id
0,U01,NIAAA,live,3,2019.0,0.0,0.0,PORTLAND,,"NAGEL, BONNIE J",...,"PROJECT SUMMARY. During young adulthood, drink...",National Consortium on Alcohol and Neurodevelo...,93429.0,HDP00632,HDP00632,HDP00632,HDP00632,9860408,9860408,NaN
1,R34,NIAAA,live,3,2019.0,0.0,0.0,SANTA MONICA,kwatkins@rand.org,"WATKINS, KATHERINE E",...,PROJECT SUMMARY/ABSTRACT. Substance use disord...,Implementing Medication-Assisted Therapy for S...,99988.0,HDP00696,HDP00696,HDP00696,HDP00696,9755001,9755001,NaN
2,R01,NIAAA,live,3,2019.0,0.0,0.0,SANTA MONICA,damico@rand.org,"D'AMICO, ELIZABETH J.",...,Summary/Abstract. A recent review of trajector...,AOD Use Trajectories from Age 10 to 24: Multi-...,182670.0,HDP00509,HDP00509,HDP00509,HDP00509,9850412,9850412,NaN
3,R34,NIAAA,live,1,2024.0,0.0,0.0,ANN ARBOR,acfernan@med.umich.ed,"FERNANDEZ, ANNE CHRISTIE",...,Project Summary/Abstract. Opioid agonist thera...,Cultivating Recovery: A Pilot Study of Digital...,390000.0,HDP01356,HDP01356,HDP01356,HDP01356,10996795,10996795,NaN
4,R01,NIAAA,live,1,2024.0,0.0,0.0,MAYWOOD,fqeadan@luc.edu,"QEADAN, FARES",...,Project Summary/Abstract. The intersection of ...,Advancing Integrated Treatment for Co-Occurrin...,385000.0,HDP01380,HDP01380,HDP01380,HDP01380,11001407,11001407,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,NaN,NaN,live,0,NaN,0.0,NaN,NaN,NaN,NaN,...,"Randomized, Double-Blind, Placebo-Controlled T...",,0.0,NaN,NaN,HDP01320,HDP01320,NaN,NaN,HDP01320
1315,NaN,NaN,live,0,NaN,76.9,NaN,NaN,NaN,NaN,...,Culturally Centered MAT for OUD Implementation...,,0.0,NaN,NaN,HDP01321,HDP01321,NaN,NaN,HDP01321
1316,NaN,NaN,live,0,NaN,63.5,NaN,NaN,NaN,NaN,...,Harnessing Digital Health to Understand Clinic...,,0.0,NaN,NaN,HDP01322,HDP01322,NaN,NaN,HDP01322
1317,NaN,NaN,live,0,NaN,0.0,NaN,NaN,NaN,NaN,...,Individual Level Predictive Modeling of Opioid...,,0.0,NaN,NaN,HDP01323,HDP01323,NaN,NaN,HDP01323


In [16]:
print("==== Frequencies of several research networks in the combined dataset ========")
mysql_fields_resnet['Research Network'].value_counts()

==== Frequencies of several research networks in the combined dataset ========


Research Network
                       1237
CTN                     199
HBCD                    159
JCOIN                    84
BACPAC                   68
ACT NOW                  45
EPPIC-NET                42
PRISM                    31
HPC                      28
DATA 2 ACTION            27
HEALING COMMUNITIES      23
HARM REDUCTION           15
HOPE                     14
IMPOWR                   14
RE-JOIN                   7
A2CPS                     5
PRECISION                 5
MIRHIQL                   5
PAIN ERN                  4
MFMU                      3
Name: count, dtype: int64

In [17]:
## Find out which columns have NA values, and investigate for incompletemess?
print("Number of empty values for each of the fields gathered in the combined dataset:")
for k in list(all_data.columns):
    print(f"{k} : {get_na_count(all_data, k)}")

Number of empty values for each of the fields gathered in the combined dataset:
Activity Code : 41
Administering IC : 41
Archived : 104
Award Type : 0
Award Year : 40
CEDAR Form % : 104
Checklist Exempt : 41
City : 46
Contact Email : 40
Contact PI : 41
Do not Engage : 41
HEAL-Related : 43
Institution(s) : 41
NIH PO : 99
PI(s) : 0
Platform Reg Time : 1060
Project # : 0
Project End : 40
Project Start : 41
Repo per Platform : 1228
Reporter Link : 40
Research Focus : 61
Research Network : 0
Research Program : 173
SBIR/STTR : 41
State : 48
Summary : 0
Title : 0
Total Funded : 0
hdp_id_x : 144
hdp_id_y : 144
key : 0
study_hdp_id : 104
study_hdp_id_appl : 144
study_most_recent_appl : 40
hdp_id : 1434


In [18]:
combined_data = all_data.copy(deep=True)

In [19]:
### Final manipulation of the combined data to make it Monday board ready

## Adding a study_type column to indicate types of entries. This will be used to put rows into their respective categories on Monday Board
combined_data['study_type'] = [ 'CTN' if m.startswith('CTN') else ('APPLIDONLY' if pd.isna(k) else 'HDP') for (m,k) in combined_data[['Project #', 'study_hdp_id_appl']].values]
print("Counts for study types in the final dataset")
print(combined_data.study_type.value_counts())

## Create a column named "Location"
from datetime import datetime
combined_data['City'] = combined_data[['City']].fillna('-')
combined_data['State'] = combined_data[['State']].fillna('-')
combined_data['Location'] = [c+","+s for (c,s) in combined_data[['City', "State"]].values]

## Convert dates to ISO format
combined_data['Project Start'] = pd.to_datetime(combined_data['Project Start'], format='%Y-%m-%d', errors='coerce').dt.date
combined_data['Project End'] = pd.to_datetime(combined_data['Project End'], format='%Y-%m-%d', errors='coerce').dt.date
combined_data['Platform Reg Time'] = pd.to_datetime(combined_data['Platform Reg Time'], utc=True).dt.date

## Change archived column to have "archived/n" values and Y/N type values in HEAL-related and SBIR/STTR columns
combined_data['Archived'] = [a if a=='archived' else 'n' for a in combined_data['Archived']]
combined_data['HEAL-Related'] = ['Y' if ((p != 'CTN' ) and (pd.isna(a))) else 'N' for (p,a) in combined_data[['study_type', 'HEAL-Related']].values]
combined_data['SBIR/STTR'] = ['Y' if 'SBIR/STTR'==t else 'N' for t in combined_data['SBIR/STTR']]
combined_data['Checklist Exempt'] = ['Y' if 1==t else 'N' for t in combined_data['Checklist Exempt']]
combined_data['Do not Engage'] = ['Y' if 1==t else 'N' for t in combined_data['Do not Engage']]

## Rename a few of the other columns:
combined_data.rename(columns={'study_most_recent_appl':'Most Recent Appl_ID', 'study_hdp_id_appl':'HDP appl_ID'}, inplace=True)
combined_data.drop(columns=['study_hdp_id', 'hdp_id', 'hdp_id_x', 'hdp_id_y'], inplace=True)


Counts for study types in the final dataset
study_type
HDP           1330
APPLIDONLY     104
CTN             40
Name: count, dtype: int64


In [20]:
handled_columns = ['study_type', 'City', 'State', 'Location', 'Project Start', 'Project End', 'Platform Reg Time', 'Archived', 'HEAL-Related', 'SBIR/STTR', 'Checklist Exempt', 'Do not Engage']
rest_obj_columns = [k for k in combined_data.columns if k not in handled_columns and combined_data[k].dtype in ['object', 'str']]

#combined_data[rest_obj_columns].fillna('-')
print("Setting empty cells to  '-' in the following colulmns:")
print(rest_obj_columns)

for k in rest_obj_columns:
    combined_data[k] = ['-' if (t is np.nan) or (t=='') else t for t in combined_data[k]]


Setting empty cells to  '-' in the following colulmns:
['Activity Code', 'Administering IC', 'Award Type', 'Contact Email', 'Contact PI', 'Institution(s)', 'NIH PO', 'PI(s)', 'Project #', 'Repo per Platform', 'Reporter Link', 'Research Focus', 'Research Network', 'Research Program', 'Summary', 'Title', 'key', 'HDP appl_ID', 'Most Recent Appl_ID']


In [21]:
#TODO: Find what's in Monday.com board, but not in mysql extract
# Mark these entries for deletion, and these would have to be deleted manually on Monday.com
### A few checks:
## How many of the "keys" from Monday board are in lookup fields?
print(f"Number records from Monday already in final dataset: {len(monday_board[monday_board.Name.isin(combined_data.key)])}")
## How many of the keys from MOnday board are not there in looup fields
mondayboard_missing_in_data = monday_board[~monday_board.Name.isin(combined_data.key)]
print(f"Number records from Monday that are not in lookup table (Consider these as discrepancies **Investigate**): {len(mondayboard_missing_in_data)}")
## How many of the keys from lookup fields are not there in Monday??
data_missing_in_mondayboard = combined_data[~combined_data.key.isin(monday_board.Name)]
print(f"Number records from lookup table that are not on Monday (Potentially new entries): {len(data_missing_in_mondayboard)}")


Number records from Monday already in final dataset: 1459
Number records from Monday that are not in lookup table (Consider these as discrepancies **Investigate**): 4
Number records from lookup table that are not on Monday (Potentially new entries): 15


In [22]:
print("****** Investigate/Delete the following entries on Monday that are not in there in the new Monday Excel upload")
mondayboard_missing_in_data

****** Investigate/Delete the following entries on Monday that are not in there in the new Monday Excel upload


,Name,Most Recent Appl_ID,HDP appl_ID,Project #,Archived,HEAL-Related,Research Focus,Research Program,Title,Contact PI,...,"""Get the Data"" Engagement Board",Data Engagement,Repo per Platform,Platform Reg Time,CEDAR Form %,Creation Log,study_type,DD Tracker,Checklist Exempt,Do not Engage
1363,10428343_HDP00882,NaN,NaN,75N95020P00589-P00001-0-1,NaN,NaN,NaN,NaN,MEDICAL WRITING CONSULTING AND DRUG DOSSIER DE...,"Sigman, Caroline",...,NaN,NaN,NaN,NaN,NaN,"Kathy Jooss May 4, 2023 11:25 AM",NaN,NaN,NaN,NaN
1364,10488140_HDP00883,NaN,NaN,75N95019D00026-P00003-759501900088-1,NaN,NaN,NaN,NaN,SCREENING OF INVESTIGATIONAL AGENTS THROUGH TH...,"Leahy, Emer",...,NaN,NaN,NaN,NaN,NaN,"Kathy Jooss May 4, 2023 11:25 AM",NaN,NaN,NaN,NaN
1365,9673173_none,NaN,NaN,5U24HD095254-02,NaN,NaN,Enhanced Outcomes for Infants and Children Exp...,NaN,ACT NOW Clinical Trials: ESC and Weaning Proto...,"DAS, ABHIK",...,NaN,NaN,NaN,NaN,NaN,"Kathy Jooss Nov 29, 2021 8:34 AM",NaN,NaN,NaN,NaN
1366,9769689_none,NaN,NaN,5R01DE027454-02,NaN,NaN,Preclinical and Translational Research in Pain...,NaN,Modeling temporomandibular joint disorders pai...,"CHEN, YONG",...,NaN,Yong Chen,NaN,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,NaN,NaN,NaN


In [23]:

#having an index column is created as a temporary column and is integral to the QA process. See SOP for more specific notes.
combined_data.index.name = 'index'

outfile = input_dir/"MondayBoard_Update.xlsx"
print(f"Exporting data to excel file at {outfile}")
combined_data.to_excel(outfile, engine='xlsxwriter')

print("Making sure uniqueness of key values. Min and Max should both be 1")
key_counts = combined_data.groupby('key').size()
key_counts.describe()

Exporting data to excel file at /Users/hinashah/Documents/HEAL/MondayUpdate_Feb2025/MondayBoard_Update.xlsx
Making sure uniqueness of key values. Min and Max should both be 1


count    1474.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
dtype: float64

*******************
*******************
DEBUG CODE BELOW
*******************
*******************

In [55]:
combined_data.to_csv("/tmp/MondayBoard.csv", index=False)

In [ ]:
combined_data[combined_data['key']=='HDP00889'].drop_duplicates()

,Most Recent Appl_ID,HDP appl_ID,key,Research Focus,Research Program,DAI Import Status,HEAL-Related,Project #,Title,Contact PI,...,repo_22_2,repo_22_3,Platform Reg Time,CEDAR Form %,Repo per Platform,Archived,Network,Contact Email,Location,study_type
index,,,,,,,,,,,,,,,,,,,,,
660,10593312,10601172,HDP00889,NaN,NaN,NaN,Y,3R24DA055306-02S1,Wake Forest IMPOWR Dissemination Education and...,"ADAMS, MEREDITH C. B.",...,NaN,NaN,2022-07-26,5.8,NaN,,NaN,meradams@wakehealth.edu,"WINSTON-SALEM,NC",HDP


In [ ]:
comparison_cols = ['Most Recent Appl_ID', 'HDP appl_ID', 'Contact Email', 'Network']

comparison_df = pd.merge(monday_board[['Name'] +comparison_cols ], combined_data[['key'] + comparison_cols], left_on = 'Name', right_on='key').drop_duplicates()
comparison_df.to_csv(input_dir/"comparison.csv", index=False)
comparison_df.to_excel(input_dir/"comparison.xlsx", index=False)

In [ ]:
pi_emails_df_updated[ pi_emails_df_updated.study_most_recent_appl=='9901704']

,study_most_recent_appl,pi_email
881,9901704,jhambm@upmc.edu


In [ ]:
resnet_df[resnet_df.appl_id=='9908734']

,appl_id,res_net,res_net_override_flag
1486,9908734,NaN,0
